In [1]:
import numpy as np
import gymnasium as gym

C:\Users\ian\anaconda3\lib\site-packages\gymnasium\envs\registration.py:521: UserWarning: WARN: Overriding environment GymV26Environment-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
C:\Users\ian\anaconda3\lib\site-packages\gymnasium\envs\registration.py:521: UserWarning: WARN: Overriding environment GymV22Environment-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [2]:
env = gym.make('FrozenLake-v1')
env.reset()

(0, {'prob': 1})

In [3]:
nS = env.observation_space.n
nA = env.action_space.n

In [4]:
env.P


{0: {0: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  2: [(0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)],
  3: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 1: {0: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False)],
  2: [(0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  3: [(0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 2:

In [7]:
def policy_eval(policy, env, discount_factor = 1.0, theta = 0.00001):
    V = np.zeros(nS)
    V1 = np.copy(V)
    while True:
        delta = 0
        for s in range(nS):
            v = 0
            for a, action_prob in enumerate(policy[s]):
                for prob, next_state, reward, done in env.P[s][a]:
                    v += action_prob * prob * (reward + discount_factor * V[next_state])
            delta = max(delta, np.abs(v - V[s]))
            V1[s] = v
        V = np.copy(V1)
        
        if delta < theta:
            break
    return np.array(V)

In [8]:
random_policy = np.ones([env.observation_space.n, env.action_space.n]) /  env.action_space.n
v = policy_eval(random_policy, env)
print('狀態值函數: ')
print(v.reshape((int(nS ** 0.5), int(nS ** 0.5))))

狀態值函數: 
[[0.01390102 0.01160346 0.02092871 0.01045776]
 [0.0162259  0.         0.04074201 0.        ]
 [0.03479274 0.08816183 0.14204605 0.        ]
 [0.         0.1758143  0.43928541 0.        ]]


In [10]:
def policy_improvement(env, policy_eval_fn = policy_eval, discount_factor = 1.):
    def one_step_lookahead(state, V):
        A = np.zeros(nA)
        for a in range(nA):
            for prob, next_state, reward, done in env.P[state][a]:
                A[a] += prob * (reward + discount_factor * V[next_state])
        return A
    
    policy = np.ones([nS, nA]) / nA #random policy
    
    while True:
        V = policy_eval_fn(policy, env, discount_factor)
        
        policy_stable = True
        
        for s in range(nS):
            chosen_a = np.argmax(policy[s])
            action_values = one_step_lookahead(s, V)
            best_a = np.argmax(action_values)
            if chosen_a != best_a:
                policy_stable = False
            policy[s] = np.eye(nA)[best_a]
            
        if policy_stable:
            return policy, V

In [11]:
policy, v = policy_improvement(env)


In [13]:
print("策略機率分配:")
print(policy)
print("")

print("4x4 策略機率分配 (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), (int(nS ** 0.5), int(nS ** 0.5))))
print("")

print("4x4 狀態值函數:")
print(v.reshape((int(nS ** 0.5), int(nS ** 0.5))))

策略機率分配:
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]

4x4 策略機率分配 (0=up, 1=right, 2=down, 3=left):
[[0 3 3 3]
 [0 0 0 0]
 [3 1 0 0]
 [0 2 1 0]]

4x4 狀態值函數:
[[0.8232943  0.82321549 0.82315953 0.8231305 ]
 [0.82331142 0.         0.52924366 0.        ]
 [0.8233444  0.82339085 0.76458368 0.        ]
 [0.         0.88225455 0.94112542 0.        ]]
